In [1]:
import emmental
from emmental.modules.identity_module import IdentityModule
from emmental.task import Task
from emmental.emmental_model import EmmentalModel
from torch import nn
import torch

In [2]:
def print_params(task):
    for key in task.module_pool.keys():
        print(key)
        for param in task.module_pool[key].named_parameters():
            print(param)

In [3]:
def print_params_all(task):
    for key in task.module_pool.keys():
        print(key)
        for param in task.module_pool[key].named_parameters():
            print(param)

In [4]:
task1 = Task(
    name="linear1",
    module_pool=nn.ModuleDict(
        {
            "im1": IdentityModule(),
            "linear1": nn.Linear(4, 2, bias=False),
            "linear2": nn.Linear(2, 1, bias=False),
        }
    ),
    task_flow=[
        {"module": "linear1", "inputs": [(0, 0)]},
        {"module": "linear2", "inputs": [(1, 0)]},
    ],
    loss_func=None,
    output_func=None,
)

In [5]:
task2 = Task(
    name="linear2",
    module_pool=nn.ModuleDict(
        {"im2": IdentityModule(), "linear1": nn.Linear(4, 2, bias=False)}
    ),
    task_flow=None,
    loss_func=None,
    output_func=None,
)

In [6]:
print_params(task1)
print_params(task2)

im1
linear1
('weight', Parameter containing:
tensor([[-0.1828, -0.1443, -0.2745, -0.4306],
        [ 0.4322, -0.0427,  0.1007,  0.4297]], requires_grad=True))
linear2
('weight', Parameter containing:
tensor([[0.5048, 0.4704]], requires_grad=True))
im2
linear1
('weight', Parameter containing:
tensor([[ 0.3921, -0.0760, -0.1948, -0.3949],
        [ 0.0197, -0.2435, -0.4469,  0.4664]], requires_grad=True))


In [7]:
mtl_model = EmmentalModel(name = 'all', tasks=[task1, task2])

<class 'torch.nn.modules.container.ModuleDict'>
ModuleDict()
odict_keys([])


In [8]:
mtl_model

EmmentalModel(name=all)

In [9]:
print_params(task1)
print('====')
print_params(task2)
print('====')
print_params_all(mtl_model)

im1
linear1
('weight', Parameter containing:
tensor([[-0.1828, -0.1443, -0.2745, -0.4306],
        [ 0.4322, -0.0427,  0.1007,  0.4297]], requires_grad=True))
linear2
('weight', Parameter containing:
tensor([[0.5048, 0.4704]], requires_grad=True))
====
im2
linear1
('weight', Parameter containing:
tensor([[-0.1828, -0.1443, -0.2745, -0.4306],
        [ 0.4322, -0.0427,  0.1007,  0.4297]], requires_grad=True))
====
im1
linear1
('weight', Parameter containing:
tensor([[-0.1828, -0.1443, -0.2745, -0.4306],
        [ 0.4322, -0.0427,  0.1007,  0.4297]], requires_grad=True))
linear2
('weight', Parameter containing:
tensor([[0.5048, 0.4704]], requires_grad=True))
im2


In [10]:
type(task1.module_pool)

torch.nn.modules.container.ModuleDict

In [11]:
type(task2.module_pool)

torch.nn.modules.container.ModuleDict

In [12]:
type(mtl_model.module_pool)

torch.nn.modules.container.ModuleDict

In [13]:
task_flow = [
    {"module": "linear1", "inputs": [(0, 0)]},
    {"module": "linear2", "inputs": [(1, 0)]},
]

In [14]:
var1 = torch.tensor([1.,2.,3.,4.])

In [15]:
mtl_model.module_pool['linear2'].forward(mtl_model.module_pool['linear1'].forward(var1))

tensor([-0.4093], grad_fn=<SqueezeBackward3>)

In [16]:
import numpy as np

In [22]:
m1 = np.array([[-0.1828, -0.1443, -0.2745, -0.4306], [0.4322, -0.0427, 0.1007, 0.4297]])
m1.shape

(2, 4)

In [23]:
m2 = np.array([0.5048, 0.4704])
m2.shape

(2,)

In [24]:
v1 = np.array([1.0, 2.0, 3.0, 4.0]).reshape((1,4))

In [25]:
v1 @ m1.transpose() @ m2.transpose()

array([-0.40936696])

In [26]:
mtl_model.forward(var1, task_name='linear1')

[[tensor([1., 2., 3., 4.])],
 [tensor([-3.0171,  2.3675], grad_fn=<SqueezeBackward3>)],
 [tensor([-0.4093], grad_fn=<SqueezeBackward3>)]]

In [27]:
immediate_ouput = [[var1]]
for action in task_flow:
    print(action)
    print('=========')
    input = [immediate_ouput[action_index][output_index] for action_index, output_index in action['inputs']]
    print('input', input)
    output = mtl_model.module_pool[action['module']].forward(*input)
    print('output', output)
    if isinstance(output, tuple): output = list(output)
    if not isinstance(output, list): output = [output]
    immediate_ouput.append(output)
    print(immediate_ouput)
task_flow

{'module': 'linear1', 'inputs': [(0, 0)]}
input [tensor([1., 2., 3., 4.])]
output tensor([-3.0171,  2.3675], grad_fn=<SqueezeBackward3>)
[[tensor([1., 2., 3., 4.])], [tensor([-3.0171,  2.3675], grad_fn=<SqueezeBackward3>)]]
{'module': 'linear2', 'inputs': [(1, 0)]}
input [tensor([-3.0171,  2.3675], grad_fn=<SqueezeBackward3>)]
output tensor([-0.4093], grad_fn=<SqueezeBackward3>)
[[tensor([1., 2., 3., 4.])], [tensor([-3.0171,  2.3675], grad_fn=<SqueezeBackward3>)], [tensor([-0.4093], grad_fn=<SqueezeBackward3>)]]


[{'module': 'linear1', 'inputs': [(0, 0)]},
 {'module': 'linear2', 'inputs': [(1, 0)]}]

In [ ]:
a = nn.ModuleDict({"im2": IdentityModule(), "linear1": nn.Linear(4, 2)})

In [ ]:
a.keys()

In [ ]:
a['im2']

In [ ]:
a['linear1']

In [ ]:
dir(a['linear1'].named_parameters())

In [ ]:
for i in a['im2'].named_parameters():
    print(i)